In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from hyperlib.manifold.lorentz import Lorentz
from hyperlib.manifold.poincare import Poincare
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from spektral.data import SingleLoader
from spektral.datasets.citation import Citation
from spektral.transforms import LayerPreprocess
from spektral.layers import GCNConv
from spektral.models.gcn import GCN

In [3]:
import hyperlib

In [6]:
from hyperlib.nn.layers.linear import ActivationHyperbolic, LinearHyperbolic
from hyperlib.nn.layers.graph import HGCLayer, HyperbolicAggregation, HGCNLP


In [5]:
%load_ext autoreload
%autoreload 2

In [5]:
#ConvHyperbolic(16, self.manifold, self.c0, self.c1, activation="relu")
hgc_layer = HGCLayer(100, Poincare(), 0.4, "relu")

In [8]:
from hyperlib.utils.data_utils import load_data, load_data_lp, mask_edges, process

## Load data

In [12]:
data = load_data_lp("cora", True, "resources\\data\\hgcn\\cora")

val_prop = 0.1
test_prop = 0.2
split_seed = 42
normalize_adj = False
normalize_feats = False

adj = data['adj_train']

adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_edges(
        adj, val_prop, test_prop, split_seed
)
data['adj_train'] = adj_train
data['train_edges'], data['train_edges_false'] = train_edges, train_edges_false
data['val_edges'], data['val_edges_false'] = val_edges, val_edges_false
data['test_edges'], data['test_edges_false'] = test_edges, test_edges_false
    
data['adj_train_norm'], data['features'] = process(
    data['adj_train'], data['features'], normalize_adj, normalize_feats
)


data.keys()

data["adj_train"]

data['adj_train_norm']

data["features"]

data['features']

n_nodes, feat_dim = data['features'].shape

n_nodes, feat_dim

nb_false_edges = len(data['train_edges_false'])
nb_edges = len(data['train_edges'])
nb_false_edges, nb_edges

D:\Nalex\Playgrounds\HyperbolicPackage\branch\hyperlib\hyperlib\utils\data_utils.py:198: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


(3664290, 3696)

In [20]:
data['features'].shape

torch.Size([2708, 1433])

In [30]:
hgcn = HGCNLP(1433)

In [31]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    hgcn((data['features'], data['adj_train_norm']))
    #grads = tape.gradient(loss_value, self.embedding.trainable_weights)
    #optimizer.apply_gradients(zip(grads, self.embedding.trainable_weights))

    #if step % 100 == 0:
    #    log.info("Training loss (for one batch) at step %d: %.4f"
    #        % (step, float(loss_value)))

Epoch 0


ValueError: Exception encountered when calling layer "hgc_layer_6" (type HGCLayer).

Can't convert non-rectangular Python sequence to Tensor.

Call arguments received by layer "hgc_layer_6" (type HGCLayer):
  • inputs=('tf.Tensor(shape=(2708, 1433), dtype=float32)', 'tensor(indices=tensor([[   0,    0,    0,  ..., 2706, 2707, 2707],\n                       [ 633, 1862, 2582,  ..., 1473,  165,  598]]),\n       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),\n       size=(2708, 2708), nnz=7392, layout=torch.sparse_coo)')